In [2]:
import requests
import datetime
import time
import csv

def fetch_binance_data(symbol, interval, start_time, end_time):
    BASE_URL = 'https://fapi.binance.com/fapi/v1/klines'
    limit = 1000  # maximum number of results per request
    all_klines = []

    while start_time < end_time:
        params = {
            'symbol': symbol,
            'interval': interval,
            'startTime': start_time,
            'endTime': end_time,
            'limit': limit
        }
        response = requests.get(BASE_URL, params=params)
        klines = response.json()
        if not klines:
            break
        all_klines.extend(klines)
        start_time = klines[-1][0] + 1  # start where the last batch left off
        time.sleep(1)  # wait for a second before making another request

    return all_klines

def save_to_csv(data, filename):
    with open(filename, 'w', newline='') as csvfile:
        writer = csv.writer(csvfile)
        writer.writerow(['timestamp', 'time', 'open', 'high', 'low', 'close', 'volume'])
        for entry in data:
            time_str = datetime.datetime.fromtimestamp(entry[0] / 1000).strftime('%Y-%m-%d %H:%M:%S')
            writer.writerow([entry[0], [time_str, entry[1], entry[2], entry[3], entry[4], entry[5]])

# Parameters
symbol = 'XRPUSDT'
interval = '1h'
end_time = int(time.time() * 1000)  # current time in milliseconds
start_time = end_time - (90 * 24 * 60 * 60 * 1000)  

data = fetch_binance_data(symbol, interval, start_time, end_time)

# Save data to CSV
save_to_csv(data, 'XRP.csv')